# Store item demand forecasting
In this project, I will build a machine learning model to forecast 3 months of 50 items sales in 10 stores of a company. It involves supervised learning, using a labeled training set for regression.

I will use the following pipeline, based on CRISP-DM framework:

1. Define the business problem
2. Initial data understanding
3. Feature engineering and exploratory data analysis
4. Data cleaning and preprocessing
5. Modelling
6. Determine financial results

In this notebook, I will be covering steps 1 to 3 of the pipeline above. The main objective here is to assess the time series data (5 years of sales) trends, seasonal patterns and characteristics, which will guide the modelling step and provide valuable business insights. Furthermore, I will approach these steps in detail below, explaining why I am making each decision.

# 1. Business problem

We are asked to forecast sales for 50 items across 10 stores over a three-month period.

Sales forecasting for 50 items across 10 stores over a three-month period involves utilizing historical sales data, market trends, and store-specific variables to predict future sales levels. This process enables businesses to strategically manage inventory, allocate resources effectively, and optimize sales strategies for each item and store, maximizing overall revenue and profitability.

Considering everything mentioned above, the project objectives are:
1. Identify valuable business insights about sales over time, like seasonal patterns, trends and general characteristics
2. Construct a model able to accurately predict the sales for 50 items across 10 stores over a three-month period.
3. Determine financial results given by the project.

As a result, the business problem will be resolved.